# Semantisk Kernel

I det här kodexemplet kommer du att använda [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI-ramverket för att skapa en grundläggande agent.

Målet med detta exempel är att visa dig stegen som vi senare kommer att använda i de ytterligare kodexemplen när vi implementerar de olika agentmönstren.


## Importera de nödvändiga Python-paketen


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Skapa klienten

I det här exemplet kommer vi att använda [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) för åtkomst till LLM.

`ai_model_id` är definierad som `gpt-4o-mini`. Prova att ändra modellen till en annan modell som finns tillgänglig på GitHub Models-marknadsplatsen för att se olika resultat.

För att använda `Azure Inference SDK`, som används för `base_url` för GitHub Models, kommer vi att använda `OpenAIChatCompletion`-kopplingen inom Semantic Kernel. Det finns också andra [tillgängliga kopplingar](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) för att använda Semantic Kernel med andra modellleverantörer.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Skapa agenten

Nedan skapar vi agenten som heter `TravelAgent`.

För det här exemplet använder vi mycket enkla instruktioner. Du kan ändra dessa instruktioner för att se hur agenten reagerar annorlunda.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Köra agenten

Nu kan vi köra agenten genom att definiera en tråd av typen `ChatHistoryAgentThread`. Eventuella nödvändiga systemmeddelanden tillhandahålls till agentens invoke_stream-argument `messages`.

När dessa är definierade skapar vi en `user_inputs` som representerar vad användaren skickar till agenten. I det här fallet har vi ställt in meddelandet till `Planera en solig semester åt mig`.

Du kan gärna ändra detta meddelande för att se hur agenten svarar annorlunda.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör det noteras att automatiserade översättningar kan innehålla fel eller brister. Det ursprungliga dokumentet på dess originalspråk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som kan uppstå vid användning av denna översättning.
